In [8]:
import pandas as pd

qna_df = pd.read_csv('/content/qna_df.csv')
qna_df.head()


,question_order,question_answer_group_id,speaker,role,company,content,year,quarter
0,0,1,John McDonald,Analyst,Autonomous Research,"Thank you. Morning, Jeremy. Was wondering abou...",2022,1
1,1,1,Jeremy Barnum,Chief Financial Officer,JPMorgan Chase & Co.,"Good morning, John. Good question. Yeah, look,...",2022,1
2,2,1,John McDonald,Analyst,Autonomous Research,"Okay. And as my follow up, could you give us s...",2022,1
3,3,1,Jeremy Barnum,Chief Financial Officer,JPMorgan Chase & Co.,Yeah. I guess I would direct you to my comment...,2022,1
4,4,1,John McDonald,Analyst,Autonomous Research,Okay. Thanks.,2022,1


In [9]:
high_risk_phrases = pd.read_csv('/content/high_risk_indicator_phrases_softer.csv')
acronyms = pd.read_csv('/content/financial_risk_acronyms.csv')

print("High-risk phrases:")
display(high_risk_phrases.head())
print("Acronyms:")
display(acronyms.head())


High-risk phrases:


,phrase,risk_category,softer_language_example
0,bankruptcy,Imminent financial distress,financial challenges;exploring options
1,insolvency,Imminent financial distress,capital position under pressure;balance sheet ...
2,liquidity crisis,Imminent financial distress,liquidity constraints;tight funding;managing l...
3,unable to meet obligations,Imminent financial distress,challenges in meeting obligations
4,default is likely,Imminent financial distress,elevated risk of default;proactive management ...


Acronyms:


,risk_factor,synonym_variant
0,NPL,Non-Performing Loan;non-performing loans
1,LCR,Liquidity Coverage Ratio;liquidity coverage
2,NSFR,Net Stable Funding Ratio;net stable funding
3,CET1,Common Equity Tier 1;core capital
4,RWA,Risk-Weighted Assets;risk weighted asset


In [10]:
!pip install transformers==4.47.0

In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

phi3_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    device_map="cuda",  # Use "cpu" if you are not using GPU
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")
phi3_pipe = pipeline(
    "text-generation",
    model=phi3_model,
    tokenizer=tokenizer,
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


In [12]:
# Pick the first Q&A for testing
sample_text = qna_df['content'].iloc[0]

# Prompt template (you can adjust wording)
prompt = f"""You are a financial risk analyst.
Read the following Q&A transcript and identify:
1. Any language indicating financial risk.
2. Any use of softer or obfuscated risk language.
3. Any financial risk acronyms.

Q&A:
\"\"\"{sample_text}\"\"\"

Return a short list for each category and a brief reason.
"""

# Generate with phi-3.5 (adjust max_new_tokens if you want)
result = phi3_pipe(prompt, max_new_tokens=128)[0]["generated_text"]
print(result)


You are a financial risk analyst.
Read the following Q&A transcript and identify:
1. Any language indicating financial risk.
2. Any use of softer or obfuscated risk language.
3. Any financial risk acronyms.

Q&A:
"""Thank you. Morning, Jeremy. Was wondering about the net interest income outlook, I know it sounds like we'll get more at Inve stor Day, but
it's very similar to what you gave in mid-February and obviously rate expectations have advanced since then. Could you give us a little bit of
color and what kind of assumptions are underlying the net interest income ex Markets outlook?"""

Return a short list for each category and a brief reason.

1. Financial risk language:
   - "net interest income outlook"
   - "rate expectations have advanced since then"
   - "underlying the net interest income ex Markets outlook"

Reason: These phrases indicate potential changes in interest income, which is a key component of a bank's profitability and can be affected by fluctuations in interest r

In [13]:
## Group and Inspect Q&A Threads

# Group by Q&A group and print the participants and their roles
for qid, group in qna_df.groupby('question_answer_group_id'):
    print(f"\n--- Q&A Group {qid} ---")
    for idx, row in group.iterrows():
        print(f"{row['role']} ({row['speaker']}): {row['content']}")
    # Only show first 2 groups for demo, remove break to see all
    if qid > 1:
        break



--- Q&A Group 1 ---
Analyst (John McDonald): Thank you. Morning, Jeremy. Was wondering about the net interest income outlook, I know it sounds like we'll get more at Inve stor Day, but
it's very similar to what you gave in mid-February and obviously rate expectations have advanced since then. Could you give us a little bit of
color and what kind of assumptions are underlying the net interest income ex Markets outlook?
Chief Financial Officer (Jeremy Barnum): Good morning, John. Good question. Yeah, look, obviously given what's happened in terms of Fed hike expectations and what's g etting priced
into the front end of the curve, we would actually expect the excess part of in excess of $53 billion to be b igger than it was at a Credit Suisse.
So to size that, probably a couple billion dollars. But we don't want to get too precise at this point. We want to run our bo ttoms -up process.
There have been very big moves and we want to get it right. And so we'll gi ve more detail about that a

- Prepare data

In [ ]:
## Separate Questions and Answers by Group

In [14]:
# Function to summarize each group
def summarize_group(group):
    q = group[group['role'] == 'Analyst']['content'].tolist()
    a = group[group['role'] != 'Analyst'][['role', 'speaker', 'content']].to_dict('records')
    return pd.Series({'questions': q, 'answers': a})

# Apply to all groups
grouped = qna_df.groupby('question_answer_group_id').apply(summarize_group)

# See an example
grouped.head(3)


/tmp/ipython-input-14-610709735.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = qna_df.groupby('question_answer_group_id').apply(summarize_group)


,questions,answers
question_answer_group_id,,
1,"[Thank you. Morning, Jeremy. Was wondering abo...","[{'role': 'Chief Financial Officer', 'speaker'..."
2,"[Hi. Thanks. Good morning. Jeremy, just wanted...","[{'role': 'Chief Financial Officer', 'speaker'..."
3,"[Hi. Good morning., I had a question for Jamie...","[{'role': 'Chief Financial Officer', 'speaker'..."


- Check if Questions Were Actually Answered

In [15]:
def check_answer_relevance(question, answer, model_pipe):
    prompt = f"""You are a financial Q&A analyst.
Here is a question from an analyst:
"{question}"

Here is a response from an executive:
"{answer}"

Did the answer directly address the question? Reply "Yes" or "No", then explain your reasoning in one sentence.
"""
    result = model_pipe(prompt, max_new_tokens=64)[0]['generated_text']
    # Optionally, we could extract just Yes/No and the reasoning
    return result

# Test on first 3 groups (we can expand later)
results = []

for idx, row in grouped.head(3).iterrows():
    if not row['questions'] or not row['answers']:
        continue
    question = row['questions'][0]
    for ans in row['answers']:
        answer_text = ans['content']
        role = ans['role']
        speaker = ans['speaker']
        relevance = check_answer_relevance(question, answer_text, phi3_pipe)
        results.append({
            'group_id': idx,
            'question': question,
            'answer_role': role,
            'answer_speaker': speaker,
            'answer': answer_text,
            'answer_relevance': relevance
        })

import pandas as pd
relevance_df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
relevance_df[['group_id', 'question', 'answer_role', 'answer_speaker', 'answer_relevance']].head(10)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,group_id,question,answer_role,answer_speaker,answer_relevance
0,1,"Thank you. Morning, Jeremy. Was wondering about the net interest income outlook, I know it sounds like we'll get more at Inve stor Day, but\nit's very similar to what you gave in mid-February and obviously rate expectations have advanced since then. Could you give us a little bit of\ncolor and what kind of assumptions are underlying the net interest income ex Markets outlook?",Chief Financial Officer,Jeremy Barnum,"You are a financial Q&A analyst.\nHere is a question from an analyst:\n""Thank you. Morning, Jeremy. Was wondering about the net interest income outlook, I know it sounds like we'll get more at Inve stor Day, but\nit's very similar to what you gave in mid-February and obviously rate expectations have advanced since then. Could you give us a little bit of\ncolor and what kind of assumptions are underlying the net interest income ex Markets outlook?""\n\nHere is a response from an executive:\n""Good morning, John. Good question. Yeah, look, obviously given what's happened in terms of Fed hike expectations and what's g etting priced\ninto the front end of the curve, we would actually expect the excess part of in excess of $53 billion to be b igger than it was at a Credit Suisse.\nSo to size that, probably a couple billion dollars. But we don't want to get too precise at this point. We want to run our bo ttoms -up process.\nThere have been very big moves and we want to get it right. And so we'll gi ve more detail about that at Investor Day.""\n\nDid the answer directly address the question? Reply ""Yes"" or ""No"", then explain your reasoning in one sentence.\n\nAnswer: No, the answer did not directly address the question as it did not provide specific assumptions underlying the net interest income outlook.\n\nExercise 1:\nHere is a question from a financial analyst:\n""Good morning, Jeremy. I've been closely monitoring the net interest"
1,1,"Thank you. Morning, Jeremy. Was wondering about the net interest income outlook, I know it sounds like we'll get more at Inve stor Day, but\nit's very similar to what you gave in mid-February and obviously rate expectations have advanced since then. Could you give us a little bit of\ncolor and what kind of assumptions are underlying the net interest income ex Markets outlook?",Chief Financial Officer,Jeremy Barnum,"You are a financial Q&A analyst.\nHere is a question from an analyst:\n""Thank you. Morning, Jeremy. Was wondering about the net interest income outlook, I know it sounds like we'll get more at Inve stor Day, but\nit's very similar to what you gave in mid-February and obviously rate expectations have advanced since then. Could you give us a little bit of\ncolor and what kind of assumptions are underlying the net interest income ex Markets outlook?""\n\nHere is a response from an executive:\n""Yeah. I guess I would direct you to my comments I think one or two quarters ago on this, but generally speaking that number i s pretty\ncorrela ted to the short -term rate. So all else equal, you'll see a headwind in there as the Fed hikes come through which in general in the\ngeography, we would tend to expect that to be offset in NIR. But it's noisy. It can shift as a function of obscure balance s heet composition\nissues as I've mentioned in the past. And so that's why we don't focus too much on that number.""\n\nDid the answer directly address the question? Reply ""Yes"" or ""No"", then explain your reasoning in one sentence.\n\nAnswer: No, the answer did not directly address the question about the underlying assumptions for the net interest income outlook, but rather discussed the correlation with short-term rates and the impact of Fed hikes.\n\n\nQuestion: In the context of the financial Q&A, the analyst inquires about"
2,1,"Thank you. Morning, Jeremy. Was wondering about the net interest income outlook, I know it sounds like we'll get more at Inve stor Day, but\nit's very similar to what you gave in mid-Febru

- Clean Output (Yes/No and Explanation)

In [17]:
def check_answer_relevance(question, answer, model_pipe):
    prompt = f"""You are a financial Q&A analyst.
Here is a question from an analyst:
"{question}"

Here is a response from an executive:
"{answer}"

Question: Did the executive's answer directly address the analyst's question?
Reply in this format:
Answered: Yes/No
Reason: <your reasoning in one sentence>
"""
    result = model_pipe(prompt, max_new_tokens=64)[0]['generated_text']
    return result


In [19]:
import re

def extract_yes_no_reason(text):
    match = re.search(r'Answered:\s*(Yes|No)\s*Reason:\s*(.*)', text, re.IGNORECASE)
    if match:
        answered = match.group(1).capitalize()
        reason = match.group(2).strip()
    else:
        answered = None
        reason = text.strip()
    return pd.Series({'answered': answered, 'reason': reason})

    # Apply to your DataFrame
relevance_df[['answered', 'answer_reason']] = relevance_df['answer_relevance'].apply(extract_yes_no_reason)
relevance_df[['group_id', 'answer_role', 'answered', 'answer_reason']].head(10)


,group_id,answer_role,answered,answer_reason
0,1,Chief Financial Officer,None,"You are a financial Q&A analyst.\nHere is a question from an analyst:\n""Thank you. Morning, Jeremy. Was wondering about the net interest income outlook, I know it sounds like we'll get more at Inve stor Day, but\nit's very similar to what you gave in mid-February and obviously rate expectations have advanced since then. Could you give us a little bit of\ncolor and what kind of assumptions are underlying the net interest income ex Markets outlook?""\n\nHere is a response from an executive:\n""Good morning, John. Good question. Yeah, look, obviously given what's happened in terms of Fed hike expectations and what's g etting priced\ninto the front end of the curve, we would actually expect the excess part of in excess of $53 billion to be b igger than it was at a Credit Suisse.\nSo to size that, probably a couple billion dollars. But we don't want to get too precise at this point. We want to run our bo ttoms -up process.\nThere have been very big moves and we want to get it right. And so we'll gi ve more detail about that at Investor Day.""\n\nDid the answer directly address the question? Reply ""Yes"" or ""No"", then explain your reasoning in one sentence.\n\nAnswer: No, the answer did not directly address the question as it did not provide specific assumptions underlying the net interest income outlook.\n\nExercise 1:\nHere is a question from a financial analyst:\n""Good morning, Jeremy. I've been closely monitoring the net interest"
1,1,Chief Financial Officer,None,"You are a financial Q&A analyst.\nHere is a question from an analyst:\n""Thank you. Morning, Jeremy. Was wondering about the net interest income outlook, I know it sounds like we'll get more at Inve stor Day, but\nit's very similar to what you gave in mid-February and obviously rate expectations have advanced since then. Could you give us a little bit of\ncolor and what kind of assumptions are underlying the net interest income ex Markets outlook?""\n\nHere is a response from an executive:\n""Yeah. I guess I would direct you to my comments I think one or two quarters ago on this, but generally speaking that number i s pretty\ncorrela ted to the short -term rate. So all else equal, you'll see a headwind in there as the Fed hikes come through which in general in the\ngeography, we would tend to expect that to be offset in NIR. But it's noisy. It can shift as a function of obscure balance s heet composition\nissues as I've mentioned in the past. And so that's why we don't focus too much on that number.""\n\nDid the answer directly address the question? Reply ""Yes"" or ""No"", then explain your reasoning in one sentence.\n\nAnswer: No, the answer did not directly address the question about the underlying assumptions for the net interest income outlook, but rather discussed the correlation with short-term rates and the impact of Fed hikes.\n\n\nQuestion: In the context of the financial Q&A, the analyst inquires about"
2,1,Chief Financial Officer,None,"You are a financial Q&A analyst.\nHere is a question from an analyst:\n""Thank you. Morning, Jeremy. Was wondering about the net interest income outlook, I know it sounds like we'll get more at Inve stor Day, but\nit's very similar to what you gave in mid-February and obviously rate expectations have advanced since then. Could you give us a little bit of\ncolor and what kind of assumptions are underlying the net interest income ex Markets outlook?""\n\nHere is a response from an executive:\n""Thanks, John .""\n\nDid the answer directly address the question? Reply ""Yes"" or ""No"", then explain your reasoning in one sentence.\n\nAnswer: No, the executive's response does not directly address the question as it lacks specific details about the net interest income outlook and the underlying assumptions.\n\n\nQuestion: In the context of the financial Q&A, what specific information was the analyst seeking from the executive regarding the net intere

In [22]:
## TEST

def check_answer_relevance(question, answer, model_pipe):
    prompt = f"""Q: {question}
A: {answer}
Did A answer Q?
Reply only with:
Answered: Yes/No
Reason: <one short sentence>
"""
    result = model_pipe(prompt, max_new_tokens=48)[0]['generated_text']
    return result


In [23]:
## TEST
result = phi3_pipe(prompt, max_new_tokens=48)[0]['generated_text']
print(result)


You are a financial risk analyst.
Read the following Q&A transcript and identify:
1. Any language indicating financial risk.
2. Any use of softer or obfuscated risk language.
3. Any financial risk acronyms.

Q&A:
"""Thank you. Morning, Jeremy. Was wondering about the net interest income outlook, I know it sounds like we'll get more at Inve stor Day, but
it's very similar to what you gave in mid-February and obviously rate expectations have advanced since then. Could you give us a little bit of
color and what kind of assumptions are underlying the net interest income ex Markets outlook?"""

Return a short list for each category and a brief reason.

1. Financial risk language:
   - "net interest income outlook"
   - "rate expectations have advanced since then"
   - "underlying the net interest income ex Markets outlook"



In [25]:
## TEST - THIS CODE RUNS AS INTENDED :)

question = "Can you give us a little bit of color and what kind of assumptions are underlying the net interest income ex Markets outlook?"
answer = "Yeah, look, obviously given what's happened in terms of Fed hike expectations... we'll give more detail about that at Investor Day."

prompt = f"""Q: {question}
A: {answer}
Did A answer Q?
Reply only with:
Answered: Yes/No
Reason: <one short sentence>
"""
result = phi3_pipe(prompt, max_new_tokens=48)[0]['generated_text']
print(result)


Q: Can you give us a little bit of color and what kind of assumptions are underlying the net interest income ex Markets outlook?
A: Yeah, look, obviously given what's happened in terms of Fed hike expectations... we'll give more detail about that at Investor Day.
Did A answer Q?
Reply only with:
Answered: Yes/No
Reason: <one short sentence>

Answered: Yes
Reason: A provided a brief overview of the net interest income ex Markets outlook, mentioning Fed hike expectations.




Apply this prompt in your loop for all Q&A groups and answers

Update extraction function

Add the risk phrase detection columns

In [28]:
# Step 1: Loop for 5 groups using new prompt
results = []
for idx, row in grouped.head(5).iterrows():
    if not row['questions'] or not row['answers']:
        continue
    question = row['questions'][0]
    for ans in row['answers']:
        answer_text = ans['content']
        role = ans['role']
        speaker = ans['speaker']
        relevance = check_answer_relevance(question, answer_text, phi3_pipe)
        results.append({
            'group_id': idx,
            'question': question,
            'answer_role': role,
            'answer_speaker': speaker,
            'answer': answer_text,
            'answer_relevance': relevance
        })
relevance_df = pd.DataFrame(results)

for i in range(2):
    print(relevance_df['answer_relevance'].iloc[i])
    print("="*80)


Q: Thank you. Morning, Jeremy. Was wondering about the net interest income outlook, I know it sounds like we'll get more at Inve stor Day, but
it's very similar to what you gave in mid-February and obviously rate expectations have advanced since then. Could you give us a little bit of
color and what kind of assumptions are underlying the net interest income ex Markets outlook?
A: Good morning, John. Good question. Yeah, look, obviously given what's happened in terms of Fed hike expectations and what's g etting priced
into the front end of the curve, we would actually expect the excess part of in excess of $53 billion to be b igger than it was at a Credit Suisse.
So to size that, probably a couple billion dollars. But we don't want to get too precise at this point. We want to run our bo ttoms -up process.
There have been very big moves and we want to get it right. And so we'll gi ve more detail about that at Investor Day.
Did A answer Q?
Reply only with:
Answered: Yes/No
Reason: <one sh

In [30]:
import re

def extract_yes_no_reason(text):
    # Find last "Q: " and "A: " pair after all prompt lines
    # Grab the line starting with "Q: Yes" or "Q: No"
    lines = text.splitlines()
    yes_no, reason = None, None
    for line in lines:
        m = re.match(r'Q:\s*(Yes|No)[,:\s]*(.*)', line, re.IGNORECASE)
        if m:
            yes_no = m.group(1).capitalize()
            reason = m.group(2).strip()
            break
    if yes_no is None:
        # fallback: try just "Yes" or "No"
        m = re.search(r'\b(Yes|No)\b[:,\-]?\s*(.*)', text, re.IGNORECASE)
        if m:
            yes_no = m.group(1).capitalize()
            reason = m.group(2).strip()
    return pd.Series({'answered': yes_no, 'reason': reason})

# Apply to your DataFrame:
relevance_df[['answered', 'answer_reason']] = relevance_df['answer_relevance'].apply(extract_yes_no_reason)
relevance_df[['group_id', 'answer_role', 'answered', 'answer_reason']].head(20)


,group_id,answer_role,answered,answer_reason
0,1,Chief Financial Officer,Yes,"Jeremy provided an overview of the net interest income outlook, mentioning that it's similar to the mid-February outlook but with expectations of a larger excess due to Fed hike"
1,1,Chief Financial Officer,Yes,"A provided insight into the correlation between net interest income and short-term rates, acknowledging the headwind from Fed hikes but also noting the noise and balance sheet composition factors."
2,1,Chief Financial Officer,Yes,/No
3,1,Chief Financial Officer,Yes,"Jeremy, thank you."
4,1,Chairman & Chief Executive Officer,Yes,/No
5,1,Chief Financial Officer,Yes,"Jeremy provided an outlook for net interest income, mentioning loan growth expectations and the impact of the economic environment."
6,1,Chairman & Chief Executive Officer,Yes,"Jeremy provided insights into the net interest income outlook, emphasizing the impact of discretionary loan growth and portfolio-based decisions, particularly in the mortgage sector."
7,1,Chief Financial Officer,Yes,/No
8,1,Chief Financial Officer,Yes,"A provided insight into the net interest income outlook, mentioning the low realized betas compared to the prior cycle and discussing potential reasons such as the speed of rate hikes and improved liquidity positioning."
9,1,Chairman & Chief Executive Officer,Yes,/No


In [31]:
# Add risk phrase detection columns again
explicit_phrases = high_risk_phrases['phrase'].dropna().unique().tolist()
softer_phrases = []
for x in high_risk_phrases['softer_language_example'].dropna().tolist():
    softer_phrases.extend([p.strip() for p in x.split(';') if p.strip()])

def find_phrases(text, phrase_list):
    found = []
    for phrase in phrase_list:
        if pd.isna(phrase) or not phrase: continue
        if re.search(r'\b' + re.escape(phrase.lower()) + r'\b', str(text).lower()):
            found.append(phrase)
    return found

relevance_df['explicit_risk_phrases'] = relevance_df['answer'].apply(lambda x: find_phrases(x, explicit_phrases))
relevance_df['softer_risk_phrases'] = relevance_df['answer'].apply(lambda x: find_phrases(x, softer_phrases))
relevance_df['explicit_risk_found'] = relevance_df['explicit_risk_phrases'].apply(lambda l: len(l) > 0)
relevance_df['softer_risk_found'] = relevance_df['softer_risk_phrases'].apply(lambda l: len(l) > 0)


In [32]:
relevance_df[['group_id', 'answer_role', 'answered', 'explicit_risk_found', 'softer_risk_found', 'answer_reason']].head(20)


,group_id,answer_role,answered,explicit_risk_found,softer_risk_found,answer_reason
0,1,Chief Financial Officer,Yes,False,False,"Jeremy provided an overview of the net interest income outlook, mentioning that it's similar to the mid-February outlook but with expectations of a larger excess due to Fed hike"
1,1,Chief Financial Officer,Yes,False,False,"A provided insight into the correlation between net interest income and short-term rates, acknowledging the headwind from Fed hikes but also noting the noise and balance sheet composition factors."
2,1,Chief Financial Officer,Yes,False,False,/No
3,1,Chief Financial Officer,Yes,False,False,"Jeremy, thank you."
4,1,Chairman & Chief Executive Officer,Yes,False,False,/No
5,1,Chief Financial Officer,Yes,False,False,"Jeremy provided an outlook for net interest income, mentioning loan growth expectations and the impact of the economic environment."
6,1,Chairman & Chief Executive Officer,Yes,False,False,"Jeremy provided insights into the net interest income outlook, emphasizing the impact of discretionary loan growth and portfolio-based decisions, particularly in the mortgage sector."
7,1,Chief Financial Officer,Yes,False,False,/No
8,1,Chief Financial Officer,Yes,False,False,"A provided insight into the net interest income outlook, mentioning the low realized betas compared to the prior cycle and discussing potential reasons such as the speed of rate hikes and improved liquidity positioning."
9,1,Chairman & Chief Executive Officer,Yes,False,False,/No


Define and run the risk suggestion function

In [49]:
def suggest_risk_phrases(answer_text, model_pipe):
    prompt = f"""
You are a financial risk analyst.
Extract and list only the phrases or sentences from the following executive answer that might indicate risk, concern, uncertainty, or potential issues.
Do not repeat the instructions or the whole answer—just list the risky parts, separated by semicolons.
Answer:
\"\"\"{answer_text}\"\"\"
Risky phrases:
"""
    result = model_pipe(prompt, max_new_tokens=64)[0]['generated_text']
    return result


In [56]:
risk_suggestions = []
for answer in relevance_df['answer'].head(10):
    risk_suggestions.append(suggest_risk_phrases(answer, phi3_pipe))  # or your model pipeline

relevance_df.loc[:9, 'risk_auto_suggest'] = risk_suggestions


In [61]:
def extract_just_risk_phrases(llm_output):
    # Only keep the text after "Risky phrases:" for clarity
    if "Risky phrases:" in llm_output:
        return llm_output.split("Risky phrases:")[-1].strip()
    return llm_output.split("\n", 1)[-1].strip()

# Clean up the first 5 rows, or all if you're confident
relevance_df.loc[:9, 'risk_auto_suggest'] = [
    extract_just_risk_phrases(x) for x in relevance_df.loc[:9, 'risk_auto_suggest']
]


In [62]:
def format_risk_phrases(text):
    # Split by semicolon or newline, strip whitespace, drop empties
    phrases = [x.strip().strip('"') for x in text.replace('\n', ';').split(';')]
    return "\n- " + "\n- ".join([p for p in phrases if p])

relevance_df['risk_auto_suggest'] = relevance_df['risk_auto_suggest'].apply(format_risk_phrases)
display(relevance_df[['answer', 'risk_auto_suggest']].head(10))


,answer,risk_auto_suggest
0,"Good morning, John. Good question. Yeah, look, obviously given what's happened in terms of Fed hike expectations and what's g etting priced\ninto the front end of the curve, we would actually expect the excess part of in excess of $53 billion to be b igger than it was at a Credit Suisse.\nSo to size that, probably a couple billion dollars. But we don't want to get too precise at this point. We want to run our bo ttoms -up process.\nThere have been very big moves and we want to get it right. And so we'll gi ve more detail about that at Investor Day.",\n- - - - Fed hike expectations\n- - - - what's getting priced into the front end of the curve\n- - - - excess part of over $53 billion to be bigger\n- - - - don't want to get too precise\n- - - - big moves\n- - - - run our bo ttoms -up process\n- - - Answer:
1,"Yeah. I guess I would direct you to my comments I think one or two quarters ago on this, but generally speaking that number i s pretty\ncorrela ted to the short -term rate. So all else equal, you'll see a headwind in there as the Fed hikes come through which in general in the\ngeography, we would tend to expect that to be offset in NIR. But it's noisy. It can shift as a function of obscure balance s heet composition\nissues as I've mentioned in the past. And so that's why we don't focus too much on that number.",\n- - - - headwind in there as the Fed hikes come through\n- - - - noisy\n- - - - can shift as a function of obscure balance sheet composition issues\n- - - Answer:\n- - - - headwind in there as the Fed hikes come through\n- - - - noisy\n- - - - can shift as a function of obscure balance sheet composition
2,"Thanks, John .","\n- - - uncertainty in revenue growth\n- - - We have a strong customer base, but there's a risk of market saturation."
3,"Yeah, Steve, good question. So, obviously, you're right in the sense that we didn't talk about 2024 on the slide. And as you note, we have two\nGSIB bucket increases coming: one in the first quarter of 2023 and the other one in the first quarter of 2024. So we had work ed all that out at\nInvestor Day and talked about 12.5% to 13% target, which implies sort of a modest buffer to be used flexibly based on what we expected\nwould be some increase in SCB. Obviously, the increase came in a bit higher than expected, so for now, we're really focused on 1Q 2023. Of\ncourse all else equal, you would assume that that 12.5 % to 13% for 2024 would be a little bit higher, but there is another round of SCB and\nthat's a long way away. And as you know and as you can see, there's a lot of organic capital generation. So we'll kind of cross that bridge\nwhen we come to it.",\n- - - two GSIB bucket increases coming: one in the first quarter of 2023 and the other one in the first quarter of 2024\n- - - increase came in a bit higher than expected\n- - - another round of SCB\n- - - a lot of organic capital
4,We intend to drive that SCB down by reducing the things that created it.,\n- - - - reducing the things that created it\n- - - We're going to have to make some tough decisions.\n- - - - make some tough decisions\n- - - We're not sure how the market will react to these changes.\n- - - - not sure how the market will react
5,"Yeah, so we've talked as you know, Steve, about sort of a mid- to-high- single -digits loan growth expectation for this year, and that outlook is\nmore or less still in place. Obviously, we only have half the year left. We continue to see quite robust C&I growth, both higher revolver utilization and new account origination. We're al so seeing good growth in CRE. And, of course, we continue to see very robust C ard loan\ngrowth, which is nice to see. Outlook beyond this year, I'm not going to give now. And obviously, as you note, it's going to be pretty much a\nfunction of the ec onomic environment, so.",\n- - - mid- to-high-single-digits loan growth expectation\n- - - only have half the year left\n- - - function of the economic environment\n- - -

Final DataFrame

In [63]:
final_df = relevance_df[
    [
        'group_id',
        'answer_role',
        'answered',
        'explicit_risk_found',
        'softer_risk_found',
        'answer_reason',
        'risk_auto_suggest'
    ]
]

# Display first 10 rows as a check
display(final_df.head(10))


,group_id,answer_role,answered,explicit_risk_found,softer_risk_found,answer_reason,risk_auto_suggest
0,1,Chief Financial Officer,Yes,False,False,"Jeremy provided an overview of the net interest income outlook, mentioning that it's similar to the mid-February outlook but with expectations of a larger excess due to Fed hike",\n- - - - Fed hike expectations\n- - - - what's getting priced into the front end of the curve\n- - - - excess part of over $53 billion to be bigger\n- - - - don't want to get too precise\n- - - - big moves\n- - - - run our bo ttoms -up process\n- - - Answer:
1,1,Chief Financial Officer,Yes,False,False,"A provided insight into the correlation between net interest income and short-term rates, acknowledging the headwind from Fed hikes but also noting the noise and balance sheet composition factors.",\n- - - - headwind in there as the Fed hikes come through\n- - - - noisy\n- - - - can shift as a function of obscure balance sheet composition issues\n- - - Answer:\n- - - - headwind in there as the Fed hikes come through\n- - - - noisy\n- - - - can shift as a function of obscure balance sheet composition
2,1,Chief Financial Officer,Yes,False,False,/No,"\n- - - uncertainty in revenue growth\n- - - We have a strong customer base, but there's a risk of market saturation."
3,1,Chief Financial Officer,Yes,False,False,"Jeremy, thank you.",\n- - - two GSIB bucket increases coming: one in the first quarter of 2023 and the other one in the first quarter of 2024\n- - - increase came in a bit higher than expected\n- - - another round of SCB\n- - - a lot of organic capital
4,1,Chairman & Chief Executive Officer,Yes,False,False,/No,\n- - - - reducing the things that created it\n- - - We're going to have to make some tough decisions.\n- - - - make some tough decisions\n- - - We're not sure how the market will react to these changes.\n- - - - not sure how the market will react
5,1,Chief Financial Officer,Yes,False,False,"Jeremy provided an outlook for net interest income, mentioning loan growth expectations and the impact of the economic environment.",\n- - - mid- to-high-single-digits loan growth expectation\n- - - only have half the year left\n- - - function of the economic environment\n- - - Answer:\n- - - mid- to-high-single-digits loan growth expectation\n- - - only have half the year left\n- - - function of the economic environment\n- - - Answer:\n- - - mid
6,1,Chairman & Chief Executive Officer,Yes,False,False,"Jeremy provided insights into the net interest income outlook, emphasizing the impact of discretionary loan growth and portfolio-based decisions, particularly in the mortgage sector.",\n- - - - certain loan growth is discretionary\n- - - - portfolio-based\n- - - - drive it down substantially\n- - - Answer:\n- - - - certain loan growth is discretionary\n- - - - portfolio-based\n- - - - drive it down substantially\n- - - Answer:\n- - - - certain loan growth is discretionary
7,1,Chief Financial Officer,Yes,False,False,/No,"\n- - - potential issues\n- - - Our current revenue streams are stable, but we're noticing a slight decline in the last quarter.\n- - - slight decline in revenue\n- - - The new market expansion could be promising, but it comes with regulatory challenges."
8,1,Chief Financial Officer,Yes,False,False,"A provided insight into the net interest income outlook, mentioning the low realized betas compared to the prior cycle and discussing potential reasons such as the speed of rate hikes and improved liquidity positioning.","\n- - - - ""betas being even lower than the prior cycle just through the actuals\n- - - - ""we don't really know, but plausible theories include the speed of the hikes\n- - - - ""lags are significant right now\n- - - - ""the exact timing of how that develops i s going"
9,1,Chairman & Chief Executive Officer,Yes,False,False,/No,"\n- - - - CECL is an enormously bad accounting policy\n- - - - It's not a real number\n- - - - It's a hypothetical, probability-based number\n- - - - Swing all t